<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span><ul class="toc-item"><li><span><a href="#Подготовка.-Разделение-признаков,-автоматизация-обучения." data-toc-modified-id="Подготовка.-Разделение-признаков,-автоматизация-обучения.-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Подготовка. Разделение признаков, автоматизация обучения.</a></span></li><li><span><a href="#Обучение-модели-линейной-регрессии-на-исходной-выборке." data-toc-modified-id="Обучение-модели-линейной-регрессии-на-исходной-выборке.-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Обучение модели линейной регрессии на исходной выборке.</a></span></li><li><span><a href="#Обучение-модели-линейной-регрессии-после-умножения-признаков-на-обратимую-матрицу" data-toc-modified-id="Обучение-модели-линейной-регрессии-после-умножения-признаков-на-обратимую-матрицу-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Обучение модели линейной регрессии после умножения признаков на обратимую матрицу</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Так, ок. Пока совсем-совсем ничего не понятно, поэтому переименую столбцы, пока думаю над заданиями проекта.

In [3]:
df.columns = ['gender', 'age', 'salary', 'relatives', 'insurance_payment']
df.head()

,gender,age,salary,relatives,insurance_payment
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Отлично, теперь посмотрим инфо по датасету.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
gender               5000 non-null int64
age                  5000 non-null float64
salary               5000 non-null float64
relatives            5000 non-null int64
insurance_payment    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропусков нет, типы данных соответствуют. Отступать некуда, линал зовёт.

Ну ладно, посмотрю ещё на корреляции признаков (прокрастинируем матричные работы).

In [5]:
df.corr()

,gender,age,salary,relatives,insurance_payment
gender,1.000000,0.002074,0.014910,-0.008991,0.010140
age,0.002074,1.000000,-0.019093,-0.006692,0.651030
salary,0.014910,-0.019093,1.000000,-0.030296,-0.014963
relatives,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
insurance_payment,0.010140,0.651030,-0.014963,-0.036290,1.000000


Логичная корреляция возраста и выплат: чем дольше живет человек, тем больше вероятности произойти чему угодно и возникнуть страховой выплате, соответственно.

Ну вот, пока я всматривалась в датасет, кажется, пришло глобальное понимание задачи. 

Итак, у нас есть датасет с данными (ваш кэп), по которому можно легко изучить личную жизни каждого из застрахованных, что, возможно, мешало бы каким-то объективным действиям со стороны страховой компании. А также могло бы навредить самим клиентам в случае утечки данных. 
Грубо говоря, нам нужно, чтобы в этой табличке тоже были числа, но они были бы не похожи на возраст и зарплату, например. Для этого нужно как-то преобразовать исходники: умножить на что-то или вычесть миллион. При этом качество предсказания целевого признака не должно измениться. 


Для начала вглянем на теоретическую сторону вопроса, и посмотрим, как пример одного из преобразований признаков влияет на качество линейной регрессии.

## Умножение матриц

<b> Итак, ответим на вопрос: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?  </b>

**Ответ:** нет, не изменится




**Обоснование:** 

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Используемые в доказательстве формулы: 

Предсказания:

$$
a = Xw
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Распишем формулу для весов для исходной матрицы и для умноженной на обратимую матрицу P:

$$
w1 = (X^T X)^{-1} X^T y
$$

$$
w2 = ((XP)^T (XP))^{-1} (XP)^T y
$$

Кое-что попереносим в другие части уравнений и кое-что посокращаем:



$$
w1 X^T X = X^T y
$$

$$
w1 X = y
$$

$$
w2 (XP)^T (XP) = (XP)^T y
$$

$$
w2 X P = y
$$

Можем приравнять первое уравнение со вторым через целевой признак y, а затем вывести расчёт весов w2 через w1:

$$
w1 X = w2 X P
$$

$$
w2 = w1 / P
$$

Теперь подставим это в формулу для предказаний преобразованных признаков:

$$
a2 = X P w2 = X P w1 / P = X w1 = a1
$$

ЧТД. Предсказания идентичны, соответственно, и качество тоже.

## Алгоритм преобразования

Так как вариант умножения признаков на матрицу (причём любую) работает, и качество обучения при этом не меняется, то используем это же преобразование в нашем датасете.

**Алгоритм:**

1. Делим датасет на признаки и целевой признак. 
2. Признаки умножаем на случайно созданную обратимую матрицу размером 4 на 4.
3. Обучаем модель линейной регрессии на преобразованных признаках.
4. Считаем метрику R2 и сравниваем её с аналогичной метрикой для исходных данных.

**Обоснование:**

Мы применяем умножение на матрицу потому, что в разделе 2 доказано, что умножение признаков на любую матрицу не меняет результата предсказания. 

Размер 4 на 4 выбран потому, что при умножении матриц ширина одной должна быть равна длине другой матрицы, при этом совпадающее число "схлопывается". И, чтобы результат умножения тоже содержал 4 столбца, второе измерение для матрицы тоже задаем числом 4. Обратимой эта матрица должна быть, очевидно потому, что в формуле вычисления весов линейной регрессии понадобится обратная нашей матрица.

Перейдём к практике.

### Подготовка. Разделение признаков, автоматизация обучения.

Для начала нужно отделить целевой признак от остальных.

In [6]:
features_source = df.drop('insurance_payment', axis=1)
target = df['insurance_payment']

Так как нам предстоит экспериментировать с матрицами, а затем проверять качество обучения, неплохо бы иметь функцию, которая быстро сделает предсказания и посчитает метрику качества R2. Это позволит не проходить каждый раз шаги отделения тестовой выборки, создания модели линейной регрессии, обучения, предсказания и оценки. 

In [7]:
# ф-я делит выборку на тест (20%) и обучающую, предсказывает линейной регрессией целевой признак, выдаёт предсказание и метрику R2:
def predict_and_R2(features, target):
    features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                                target, 
                                                                                test_size=0.2, 
                                                                                random_state=42)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_test)
    r2 = r2_score(target_test, predictions)
    return predictions, r2

### Обучение модели линейной регрессии на исходной выборке.

Испробуем нашу функцию на исходных данных и посмотрим на R2.

In [8]:
predictions_source, r2_source = predict_and_R2(features_source, target)
print("R2 на исходной выборке равно:", r2_source)

R2 на исходной выборке равно: 0.4368694923137991


Метрика не отрицательная и не ноль. Неплохо. Посмотрим, что будет, если умножить матрицу с признаками на обратимую матрицу.

### Обучение модели линейной регрессии после умножения признаков на обратимую матрицу

Так как наша матрица с признаками совсем не квадратная, умножить на обратную ей не получится. Нам нужен квадратик, сторона которого равна числу признаков. Вызовем случайную обратимую матрицу и умножим на неё признаки.

In [9]:
# создаём рандомную матрицу 4 на 4, потому что у нас 4 признака, иначе нельзя будет перемножить матрицы
random_matrix = np.random.rand(4,4)
# проверяем, создаётся ли обратная матрица без ошибок (если да, то наша матрица обратима)
np.linalg.inv(random_matrix)

# умножаем матрицу с признаками на матрицу случайную
features_processed = features_source @ random_matrix
print("\nПризнаки после преобразвания:")
print(features_processed.head())


Признаки после преобразвания:
              0             1             2             3
0  26934.445155  26443.798561  27017.816385  29858.588291
1  20646.180686  20269.775601  20712.100341  22889.850409
2  11411.918398  11203.848603  11448.982743  12652.732750
3  22635.035301  22222.539896  22702.692699  25090.182675
4  14177.728323  13919.562924  14222.772446  15717.990068


15254 члена семьи у 34230-летнего клиента. Данные спрятаны неплохо. 

Теперь посмотрим, как выглядит метрика для признаков, умноженных на случайную матрицу.

In [10]:
predictions_processed, r2_processed = predict_and_R2(features_processed, target)
print("R2 на исходной выборке равно:", r2_processed)

R2 на исходной выборке равно: 0.4368694922938444


Если прогнать две последние ячейки кода несколько раз (таким образом создавая новую случайную матрицу каждый раз), можно количественно убедиться, что метрика R2 практически идентична той, что мы получили на исходных данных, значение совпадает до 11го знака после запятой. 

Возрадуемся!

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования